In [ ]:
import pandas as pd

In [ ]:
df_data = pd.read_csv('./data.csv', index_col=0)
df_data.head(5)

In [ ]:
df_data.info()

In [ ]:
df_data['Desc']

In [ ]:
from nltk.tokenize import RegexpTokenizer

In [ ]:
text01 = 'We know that power is shifting: From West to E'
def remove_punctuation(text):
  tokenizer = RegexpTokenizer('[a-zA-Z]+')                # 이렇게 하면 리스트로 나온다. 그리고 리스트를 space로 구분된 string으로 만들어줘야 한다. 
  text_list = tokenizer.tokenize(text)
  # print(type(text_list), text_list)
  result = ' '.join(text_list)
  # print(type(result), result)
  return result

In [ ]:
remove_punctuation(text01)                  # for text01 in df_data['Desc']: 과 같은 효과

In [ ]:
df_data['cleaned'] = df_data['Desc'].apply(remove_punctuation) # apply 안에 함수를 넣어준다.

In [ ]:
df_data.columns

In [ ]:
df_data['cleaned'].head(5)

In [ ]:
df_data.info()

## Pre-Trained

In [ ]:
!curl -O https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz

In [ ]:
from gensim.models import Word2Vec

In [ ]:
corpus = []
for words in df_data['cleaned']:
  corpus .append(words.split())
print(type(corpus), corpus[4:8])

In [ ]:
word2vec_model = Word2Vec(min_count=2, workers=-1, size=300)
word2vec_model.build_vocab(corpus)

In [ ]:
word2vec_model.intersect_word2vec_format('GoogleNews-vectors-negative300.bin.gz', lockf=1.0, binary=True)

In [ ]:
word2vec_model.train(df_data['cleaned'], total_examples=word2vec_model.corpus_count, epochs=15)

In [ ]:
word2vec_model.wv.vocab

In [ ]:
doc_embedding_list = list()
# for line in df_data['cleaned'][2:3]:
for line in df_data['cleaned']:
  # print(line)
  doc2vec = None
  for word in line.split():
    # print(word)
    if word in word2vec_model.wv.vocab:
      # print(word2vec_model[word])
      if doc2vec is None:
        doc2vec = word2vec_model[word] 
      else :
        doc2vec = doc2vec + word2vec_model[word] 

  if doc2vec is not None:
    doc2vec = doc2vec / len(doc2vec)
    doc_embedding_list.append(doc2vec)
  # print(type(doc2vec), doc2vec.shape, doc2vec)

print(doc_embedding_list)

In [ ]:
type(doc_embedding_list), doc_embedding_list[3]

## cosin matrix

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
cosine_sim = cosine_similarity(doc_embedding_list, doc_embedding_list)
type(cosine_sim), cosine_sim.shape

In [ ]:
cosine_sim[3]

## Predict

In [ ]:
# title = 'The Da Vinci Code'
title = 'The Four Pillars of Investing'   # 3

In [ ]:
idx = 3
sim_scores = list(enumerate(cosine_sim[3]))
print(sim_scores)

In [ ]:
sim_scores_list = sorted(sim_scores, key=lambda x:x[1], reverse=True)

In [ ]:
sim_scores_list[1:6]

In [ ]:
df_data.head(7)